In [2]:
import numpy as np
from matplotlib import pyplot as plt

### Define Model

In [2]:
'''
Model class for the network model
Store the model, including layer(construct by node), activation function
'''
class Model():
    def __init__(self, layer_nums, create_func, input_arr, lr_rate):
        self.layer_nums = layer_nums            # List of the number of each layer
                                                # 0 => input number
                                                # middle => layer number
                                                # last => output number
                    
        self.layer_list = []                    # Store layer list
        create_func(self, input_arr, layer_nums)            # Creating the network
        
        self.result = 0                         # Initial network result
        self.lr_rate = lr_rate                  # Learning rate
        self.loss = 0

        


    '''
    Calculate the network by using input data
    '''
    def cal_network(self, input):
        self.layer_list[0].set_input(input.copy())
        for i, layer in enumerate(self.layer_list):
            layer.forwrad_pass()

        return self.get_result()

    '''
    Set output errors, for the last layer only
    '''
    def set_output_error(self, error):
        self.layer_list[len(self.layer_nums)-1].set_output_error(error)
  
    '''
    Adjust nodes in network using backpropagation and ground truth
    '''
    def adjust_model(self, ground_truth):
        error = self.result - ground_truth
        self.loss = np.dot(error, error) / 2
        if ground_truth == 1:
            error = error * 3
        self.set_output_error(error)
        for i in range(len(self.layer_list)-1, -1, -1):
            self.layer_list[i].adjust_weight(self.lr_rate)

    '''
    Return network result
    '''
    def get_result(self):
        self.result = self.layer_list[len(self.layer_nums)-1].get_output()
        return self.result

    def get_loss(self):
        return self.loss

    def get_output_w(self):
        w = self.layer_list[len(self.layer_list)-1].get_output_w()
        return w

### Define Layer

In [3]:
#@title
'''
Layer class for the network model
Help model to handle neurons
'''
class Layer_vec():
    '''
    Initial layer
    @param func - activation function
    @param d_func - diviation of activation function
    @param node_num - number of nodes in this layer
    @param last_layer - last layer's node list
    @param is_first - whether this layer is the first layer
    '''
    def __init__(self, func, d_func, node_num, last_layer, is_first):
        # Activation Functions
        self.act_func = func                                      # Activation function
        self.d_act_func = d_func                                  # Diviation of activate function

        # Input definition
        if not is_first:
            self.i_num = last_layer.get_node_num()               # Number of input node
        else:
            self.i_num = len(last_layer)                         # Number of input node
        self.input_vec = np.full(self.i_num+1, 0.0)              # Initial input passed from
        self.input_vec[self.i_num] = 1
        self.neuron_num = node_num
        self.last_layer = last_layer

        # Calculation variables
        self.w = np.random.randn(self.i_num+1, node_num) / np.sqrt(self.i_num+1)      # Initial weight

        self.bp_vec = np.full(self.neuron_num, 0.0)                   # Recieve value passed from postorier layer
        self.is_first = is_first                                  # Set to true if this node is at first layer
        self.weighted_input = np.full(self.i_num, 0.0)           # Initial weighted input, use to store the value after the weighted input are sum up
        self.result = np.full(self.neuron_num, 0.0)                   # Initial output result, equal to the value after subsituted weighted input into activation function
        self.lr_rate = 0.005                                      # Learning rate of the node
  
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        self.lr_rate = lr_rate
        # Calculate each weight for the specific previous node
        delta = self.bp_vec * self.d_act_func(self.weighted_input)    # Dimation of layer node
        delta_w = np.outer(self.input_vec, delta)
        if (not self.is_first):
            pass_v = np.dot(delta, self.w[0:len(self.w)-1, :].transpose())
            self.last_layer.pass_bp(pass_v[0:len(self.input_vec)-1])
        self.w = self.w - self.lr_rate * delta_w

    def forwrad_pass(self):
        if not self.is_first:
            self.extract_value()
        self.bp_vec = np.full(self.neuron_num, 0.0)      # Set bp value to zero, for later adjustment

        self.weighted_input = np.dot(self.input_vec, self.w)
        self.result = self.act_func(self.weighted_input)
        return self.result
    
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()
        
    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_vec = x.copy()
        if self.is_first:
            self.input_vec = np.append(self.input_vec, 1)
            
    def extract_value(self):
        self.input_vec = self.last_layer.get_output()
        self.input_vec = np.append(self.input_vec, 1)

    def get_node_num(self):
        return self.neuron_num

    def set_output_error(self, error):
        if self.neuron_num != len(error):
            print("Output layer and error doesn't match")
            return
        self.pass_bp(error)

    def get_output(self):
        return self.result
    def get_output_w(self):
        return self.w.copy()

### Activation functions

In [4]:
#@title
'''
Activation function for the network
'''
def test_act_func(x):
    return x*11

'''
ReLU
'''
def ReLU(x):
    x[x<=0] = 0
    return x.copy()

'''
Sigmoid
'''
def Sigmoid(x):
    return 1/(1+np.exp(-x))


### Diviation of Activation function

In [5]:
#@title
'''
Diviation of the activation function for the network
'''
def d_test_act_func(x):
    return x+2

'''
Diviation of ReLU
'''
def d_ReLU(x):
    x[x > 0] = 1
    x[x <= 0] = 0
    return x.copy()

'''
Diviation of Sigmoid
'''
def d_Sigmoid(x):
    s = 1/(1+np.exp(-x))
    ans = s * (1 - s)
    return ans